In [23]:
import numpy
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import  pandas as pd
import  os
from keras.models import Sequential, load_model
 

In [24]:
all_data = pd.read_csv('./data/2021MCMProblemC_DataSet.csv', encoding='utf-8', index_col='Detection Date')
# all_data
## for the first question, only keep track of the sightings in 2019 and 2020
all_data.index = pd.to_datetime(all_data.index, errors='coerce')
all_data = all_data.sort_values(by = 'Detection Date', ascending=True)
data = all_data[(all_data.index >= "2019/1/1") & (all_data['Lab Status'] == "Unverified") | (all_data['Lab Status'] == "Positive ID")]
data
train_data = data[0:2219]
test_data = data[2220:]
train_data
test_data

,GlobalID,Notes,Lab Status,Lab Comments,Submission Date,Latitude,Longitude
Detection Date,,,,,,,
2020-10-01,{D1EFAFA8-EFAC-41A1-8B51-BFD867CE97EA},"Found intact, dead on patio in White Salmon, WA",Unverified,,2020/10/1,45.734625,-121.492571
2020-10-02,{2D515E68-10B9-483B-B54E-8FBC46B15A91},very large hornet hovering by my home's gutter...,Unverified,,2020/10/2,48.917375,-122.647069
2020-10-02,{28FBCBBC-350C-405A-BCD8-5C68F5C2BF83},was around 2:30 pm it landed on my horse and t...,Unverified,Thank you for you possible hornet submission. ...,2020/10/2,48.274643,-121.660088
2020-10-02,{0437C2C3-42F1-44EB-86A3-0574C16B4227},I'm almost positive that it was based on the s...,Unverified,,2020/10/2,48.789849,-122.450922
2020-10-02,{AFE21937-9622-47E2-BF84-750DB44E424B},riding by cell phone tower saw aisian hornets ...,Unverified,,2020/10/4,47.759040,-117.495860
...,...,...,...,...,...,...,...
2020-10-21,{A2B93362-E28D-43E8-A0B6-F8AE4F3903C2},About 1 1/2 inches long on the side of the bui...,Unverified,Thanks for the submission. Please feel free to...,2020/10/21,48.892087,-122.590774
2020-10-22,{9797D62C-E3F8-42B8-8DA0-665F074A27CF},It was about 2 inches long and appeared to be ...,Unverified,,2020/10/22,48.893080,-122.643823
2020-10-28,{390FCB4C-2CED-49D6-81A4-73DFB8267C00},On a walk we observed a large hornet. It struc...,Unverified,,2020/10/3,47.343386,-122.166631


In [37]:
## https://blog.csdn.net/qq_35649669/article/details/89575949

def create_dataset(dataset,look_back=2): # 每个的滑动窗口设置为2
    dataX, dataY=[], []
    for i in range(len(dataset)-look_back):
        a=dataset[i:(i+look_back)]
        dataX.append(a)    # 记录窗口的值
        dataY.append(dataset[i+look_back]) # 记录除了前面两个以外的所有值作为正确的标签
    return np.array(dataX), np.array(dataY)


In [40]:
def trainModel(train_X, train_Y):
    '''
    trainX，trainY: 训练LSTM模型所需要的数据
    '''
    model = Sequential()
    model.add(LSTM(
        140,
        input_shape=(train_X.shape[1], train_X.shape[2]),
        return_sequences=True))
    model.add(Dropout(0.3))

    model.add(LSTM(
        140,
        return_sequences=False))
    model.add(Dropout(0.3))

    model.add(Dense(
        train_Y.shape[1]))
    model.add(Activation("relu"))

    model.compile(loss='mse', optimizer='adam')
    model.fit(train_X, train_Y, epochs=100, batch_size=64, verbose=1)

    return model


In [45]:
train_data = pd.DataFrame(train_data, columns = ['Latitude', 'Longitude'])
train_data = train_data.values
train_X, train_Y = create_dataset(train_data, 2)
model = trainModel(train_X, train_Y)
model

Epoch 1/100
35/35 [==============================] - 3s 8ms/step - loss: 8434.9658
Epoch 2/100
35/35 [==============================] - 0s 6ms/step - loss: 7841.3896
Epoch 3/100
35/35 [==============================] - 0s 6ms/step - loss: 7678.5441
Epoch 4/100
35/35 [==============================] - 0s 8ms/step - loss: 7582.3201
Epoch 5/100
35/35 [==============================] - 0s 8ms/step - loss: 7528.9612
Epoch 6/100
35/35 [==============================] - 0s 8ms/step - loss: 7488.0536
Epoch 7/100
35/35 [==============================] - 0s 8ms/step - loss: 7461.8961
Epoch 8/100
35/35 [==============================] - 0s 9ms/step - loss: 7449.7282
Epoch 9/100
35/35 [==============================] - 0s 10ms/step - loss: 7443.9500
Epoch 10/100
35/35 [==============================] - 0s 10ms/step - loss: 7431.5508
Epoch 11/100
35/35 [==============================] - 0s 9ms/step - loss: 7435.6038
Epoch 12/100
35/35 [==============================] - 0s 8ms/step - loss: 7429.7910